<a href="https://colab.research.google.com/github/amatyaagrata/ConceptandTechnologyofAI/blob/main/Workshop8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Step -1- Building a Custom Decision Tree with Information Gain:

In [3]:
import numpy as np

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        """
        Initializes the decision tree.
        Parameters:
        max_depth (int, optional): Maximum depth of the tree.
        """
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        """
        Trains the decision tree model.
        """
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        """
        Recursively builds the decision tree.
        """
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}

        if num_samples == 0 or (self.max_depth is not None and depth >= self.max_depth):
            return {'class': np.bincount(y).argmax()}

        best_info_gain = -np.inf
        best_split = None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = X[:, feature_idx] > threshold

                left_y = y[left_mask]
                right_y = y[right_mask]

                if len(left_y) == 0 or len(right_y) == 0:
                    continue

                info_gain = self._information_gain(y, left_y, right_y)

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'left_mask': left_mask,
                        'right_mask': right_mask
                    }

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        left_tree = self._build_tree(
            X[best_split['left_mask']],
            y[best_split['left_mask']],
            depth + 1
        )

        right_tree = self._build_tree(
            X[best_split['right_mask']],
            y[best_split['right_mask']],
            depth + 1
        )

        return {
            'feature_idx': best_split['feature_idx'],
            'threshold': best_split['threshold'],
            'left_tree': left_tree,
            'right_tree': right_tree
        }

    def _information_gain(self, parent, left, right):
        """
        Computes Information Gain.
        """
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)

        weighted_entropy = (
            (len(left) / len(parent)) * left_entropy +
            (len(right) / len(parent)) * right_entropy
        )

        return parent_entropy - weighted_entropy

    def _entropy(self, y):
        """
        Computes entropy.
        """
        class_probs = np.bincount(y) / len(y)
        return -np.sum(class_probs * np.log2(class_probs + 1e-9))

    def predict(self, X):
        """
        Predicts class labels for X.
        """
        return np.array([self._predict_single(x, self.tree) for x in X])

    def _predict_single(self, x, tree):
        """
        Predicts a single sample.
        """
        if 'class' in tree:
            return tree['class']

        if x[tree['feature_idx']] <= tree['threshold']:
            return self._predict_single(x, tree['left_tree'])
        else:
            return self._predict_single(x, tree['right_tree'])


Step -2- Load and Split the Iris Datasets:

In [4]:
# Necessary Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target

# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Step -3- Train and Evaluate a Custom Decision Tree:

In [5]:


# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)
# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")

Custom Decision Tree Accuracy: 1.0000


Step -4- Train and Evaluate a Scikit Learn Decision Tree:

In [6]:
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)
# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

Scikit-learn Decision Tree Accuracy: 1.0000


Step -5- Result Comparison:

In [7]:
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")

Accuracy Comparison:
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000


#Exercise - Ensemble Methods and Hyperparameter Tuning.
Using the Wine Dataset from scikit-learn
1. Implement Classification Models:
• Train a Decision Tree Classifier and a Random Forest Classifier using scikit-learn.
• Compare the models based on their F1 scores.

In [8]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
dt_preds = dt.predict(X_test)
print("Decision Tree F1:", f1_score(y_test, dt_preds, average='weighted'))

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print("Random Forest F1:", f1_score(y_test, rf_preds, average='weighted'))


Decision Tree F1: 0.9439974457215836
Random Forest F1: 1.0


#2. Hyperparameter Tuning:
• Identify three hyperparameters of the Random Forest Classifier.
• Perform hyperparameter tuning using GridSearchCV to optimize these parameters.
• Take hints from the scikit-learn documentation to guide the implementation.

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid, scoring='f1_weighted', cv=3)
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
best_rf = grid.best_estimator_
print("F1 with Best RF:", f1_score(y_test, best_rf.predict(X_test), average='weighted'))


Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
F1 with Best RF: 1.0


#3. Implement Regression Model:
• Train a Decision Tree Regressor and a Random Forest Regressor using scikit-learn.
• Identify three parameters for Random Forest Regressio and Perform hyperparameter tuning using
RandomSearchCV to optimize these parameters.

In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_reg = wine.data[:, 1:]
y_reg = wine.data[:, 0]

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Decision Tree Regressor
dt_r = DecisionTreeRegressor(random_state=42)
dt_r.fit(X_train_r, y_train_r)
dt_preds = dt_r.predict(X_test_r)
print("Decision Tree MSE:", mean_squared_error(y_test_r, dt_preds))

# Random Forest Regressor
rf_r = RandomForestRegressor(random_state=42)
rf_r.fit(X_train_r, y_train_r)
rf_preds = rf_r.predict(X_test_r)
print("Random Forest MSE:", mean_squared_error(y_test_r, rf_preds))


Decision Tree MSE: 0.31197222222222226
Random Forest MSE: 0.15426672999999946


In [11]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5],
    'min_samples_leaf': [1, 2]
}

rand_search = RandomizedSearchCV(RandomForestRegressor(random_state=42),
                                 param_distributions=params, n_iter=4, cv=3)
rand_search.fit(X_train_r, y_train_r)

best_rf_r = rand_search.best_estimator_
print("Best Parameters:", rand_search.best_params_)
print("MSE with Best RF:", mean_squared_error(y_test_r, best_rf_r.predict(X_test_r)))


Best Parameters: {'n_estimators': 50, 'min_samples_leaf': 2, 'max_depth': 5}
MSE with Best RF: 0.13569595862301947
